# Imports And Libraries

In [18]:
from square.client import Client
import os
import pandas as pd

from google_sheets import *

# import pandas as pd

# import os
import pickle

# Generic Functions

In [19]:
def get_value(key1,key2):
    try:
        return key1[key2] 
    except:
        return None

# Get Existing Subs

In [20]:
SAMPLE_SPREADSHEET_ID_input = '1K_uXqI1eiguBMr0uH3Tu7xDzePAIi3uAsDrAWukyBq8'
SAMPLE_RANGE_NAME = 'Square Subs!A:G'

exist = get_spread_sheet(SAMPLE_SPREADSHEET_ID_input, SAMPLE_RANGE_NAME)

# Get Data of most Recent Dub

In [21]:
dates = exist[exist['Date'].str.contains('T')]

most_recent_transaction_date = pd.to_datetime(dates['Date'], format='mixed').max().isoformat()

most_recent_transaction_date

'2023-09-01T13:17:42.556000+00:00'

# Get New Subs

In [22]:
client = Client(
    access_token=os.environ['SQUARE_ACCESS_TOKEN'],
    environment='production')


In [23]:
orders_api = client.orders
customers_api = client.customers

In [24]:
most_recent_transaction_date

'2023-09-01T13:17:42.556000+00:00'

In [25]:
body = {
    'location_ids': [
        'SBBJSGR24YHYB'
    ],
        'sort': {
            'sort_field': 'CREATED_AT',
            'sort_order': 'DESC'
        },
    'query': {
        'filter': {
            
            'date_time_filter': {
                'created_at': {
                    'start_at': f'{most_recent_transaction_date}',
                }
            }
        },
        'sort': {
            'sort_field': 'CREATED_AT',
            'sort_order': 'ASC'
        }
    },
   }

In [26]:
s_orders = orders_api.search_orders(body).body['orders']
len(s_orders)

98

#  Get All Invoices

In [27]:
invoices_api = client.invoices


result = invoices_api.list_invoices(
    'SBBJSGR24YHYB'
)

invoices = pd.DataFrame(result.body['invoices'])

i=1
while result.cursor:
    print(i)
    i+=1
    result = invoices_api.list_invoices(
        'SBBJSGR24YHYB',
        cursor = result.cursor
    )
invoices = pd.concat([invoices, pd.DataFrame(result.body['invoices'])])
invoices


1


,id,version,location_id,order_id,payment_requests,primary_recipient,invoice_number,title,description,public_url,status,timezone,created_at,updated_at,accepted_payment_methods,delivery_method,sale_or_service_date,store_payment_method_enabled,next_payment_amount_money
0,inv:0-ChDNGbmagWxRW5oRsCVmuUoCEIMP,3,SBBJSGR24YHYB,V3AZRX79S94Y9HfeFsEjFzvIZ9YZY,[{'uid': '09b8aab5-38a8-4417-bb4d-5f7f305f5264...,"{'customer_id': 'M339V0D1Z80PXBABRE0FGQ6R30', ...",inv-103,Test,Test,https://squareup.com/pay-invoice/inv:0-ChDNGbm...,PAID,Europe/London,2023-09-01T13:17:59Z,2023-09-01T16:35:05Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,2023-09-01,False,NaN
1,inv:0-ChAwNp6EuFWFI755gKIIZZeeEIMP,2,SBBJSGR24YHYB,XBVKDIEVMFCEDklxsrCctVgeV,[{'uid': '066e540a-7545-42c2-87af-42409339a765...,"{'customer_id': 'TC0C4DZ3Y217B8Y8A1EZ0PV4F0', ...",000209,,,https://squareup.com/pay-invoice/inv:0-ChAwNp6...,PAID,Europe/London,2023-09-01T12:32:28Z,2023-09-01T17:23:41Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,NaN,False,NaN
2,inv:0-ChCsTjhZ3h5svE84fHrsVENwEIMP,0,SBBJSGR24YHYB,NaN,[{'uid': '62e6bc46-f17b-43ae-a881-ad175b148f4c...,"{'customer_id': '7NHTHHYFZMRN12QDC32KRMK03W', ...",000208,,,https://squareup.com/pay-invoice/inv:0-ChCsTjh...,UNPAID,Europe/London,2023-09-01T10:04:07Z,2023-09-01T10:04:08Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,NaN,False,"{'amount': 1000, 'currency': 'GBP'}"
3,inv:0-ChCcqmfj4UIYlnYlJCYN_4DKEIMP,0,SBBJSGR24YHYB,NaN,[{'uid': 'a3352e48-bb95-44a8-a43a-d76d05464725...,"{'customer_id': 'CJM2Y67Q8969V1NE7VPTKXPJAM', ...",000207,,,https://squareup.com/pay-invoice/inv:0-ChCcqmf...,UNPAID,Europe/London,2023-09-01T10:01:52Z,2023-09-01T10:01:53Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,NaN,False,"{'amount': 1500, 'currency': 'GBP'}"
4,inv:0-ChAHcIhTKda3P-Sl9eIixJleEIMP,0,SBBJSGR24YHYB,NaN,[{'uid': 'c306bea7-adad-466a-b447-dda4176e6ec8...,"{'customer_id': '27WR2MP9DN477DD7YJ0QD4PT3G', ...",000206,,,https://squareup.com/pay-invoice/inv:0-ChAHcIh...,UNPAID,Europe/London,2023-08-31T19:09:52Z,2023-08-31T19:09:52Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,NaN,False,"{'amount': 500, 'currency': 'GBP'}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,inv:0-ChAxyXkM34z5Rr_hR1qcghr3EIMP,2,SBBJSGR24YHYB,J8M3Q8SQJERA16Tyz8YWTyueV,[{'uid': 'ec717fdb-a9fd-431a-9ee6-69e76da24af1...,"{'customer_id': '0HTAK3MQ28S69A5KBCTZ8HKHC0', ...",000008,,,https://squareup.com/pay-invoice/inv:0-ChAxyXk...,PAID,Europe/London,2023-06-01T17:45:39Z,2023-06-05T06:19:45Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,NaN,False,NaN
42,inv:0-ChAMb2fmrMLIoY-QsyhO607cEIMP,4,SBBJSGR24YHYB,TBKQ9UN9lcEcTA52W7S6HcpeV,[{'uid': 'dce09c3e-dc3c-4b3b-a399-313acb7090dd...,"{'customer_id': '4KEVYTZKFD01B4NE8KA0N1X8Q4', ...",000007,,,https://squareup.com/pay-invoice/inv:0-ChAMb2f...,PAID,Europe/London,2023-06-01T17:45:00Z,2023-06-02T11:03:53Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,NaN,False,NaN
43,inv:0-ChBSYVNkiKpyEQslJ1DOvhaHEIMP,3,SBBJSGR24YHYB,N0B8KtXp70Qg6VikEGxGHxveV,[{'uid': '58ed6fef-84c4-4a40-8323-94be5b2f0315...,"{'customer_id': 'C3S5WM4AS12RD0CZQ6DS8QBDN4', ...",000005,,,https://squareup.com/pay-invoice/inv:0-ChBSYVN...,PAID,Europe/London,2023-06-01T17:26:29Z,2023-06-01T17:28:31Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,NaN,False,NaN
44,inv:0-ChCP7LR57_FEOKwBMVQB0odAEIMP,2,SBBJSGR24YHYB,JMwfRq0FXCGcZqDPMquKfS6eV,[{'uid': '3891f092-3070-40e1-a0f1-6c50091dca46...,"{'customer_id': 'M2WN0A8Z056ECKKPCZ3M1GY88R', ...",000003,,,https://squareup.com/pay-invoice/inv:0-ChCP7LR...,PAID,Europe/London,2023-05-25T11:45:50Z,2023-06-01T16:07:29Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,NaN,False,NaN


# Format new subs and get customer names

In [28]:
orders_format = []
for i, order in enumerate(s_orders[:]):
    print(i, '/', len(s_orders))
    new_order = {}
    
    new_order['id']=get_value(order,'id')
    new_order['created_at']=get_value(order,'created_at')
    
    new_order['customer_id']=get_value(order,'customer_id')
    
    new_order['state']=get_value(order,'state')

    if not new_order['customer_id']:
        customer_ids = []
        try:
            for tender in order['tenders']:
                if get_value(tender,'customer_id'):
                    customer_ids.append(get_value(tender,'customer_id'))
            new_order['customer_id'] = customer_ids[0]
        except:
            pass

    if new_order['customer_id']:
        customer = customers_api.retrieve_customer(new_order['customer_id'])
    
        try:
            new_order['first_name'] = get_value(customer.body['customer'],'given_name')
            new_order['last_name'] = get_value(customer.body['customer'],'family_name')
        except:
            print(customer.body)
    else:
        invoice = invoices[invoices['order_id']==new_order['id']]
        if len(invoice)>0:
            customer = customers_api.retrieve_customer(list(invoices['primary_recipient'])[0]['customer_id'])
            try:
                new_order['first_name'] = get_value(customer.body['customer'],'given_name')
                new_order['last_name'] = get_value(customer.body['customer'],'family_name')
            except:
                print(customer.body)
            
    for item in order['line_items']:
        line_dict = new_order.copy()
        
        line_dict['variation_name']=get_value(item,'variation_name')
        line_dict['item_name']=get_value(item,'name')
        line_dict['note']=get_value(item,'note')
        line_dict['total_money']=item['total_money']['amount']/100
        orders_format.append(line_dict) 


orders_df = pd.DataFrame(orders_format)

0 / 98
1 / 98
2 / 98
3 / 98
4 / 98
5 / 98
6 / 98
7 / 98
8 / 98
9 / 98
10 / 98
11 / 98
12 / 98
13 / 98
14 / 98
15 / 98
16 / 98
17 / 98
18 / 98
19 / 98
20 / 98
21 / 98
22 / 98
23 / 98
24 / 98
25 / 98
26 / 98
27 / 98
28 / 98
29 / 98
30 / 98
31 / 98
32 / 98
33 / 98
34 / 98
35 / 98
36 / 98
37 / 98
38 / 98
39 / 98
40 / 98
41 / 98
42 / 98
43 / 98
44 / 98
45 / 98
46 / 98
47 / 98
48 / 98
49 / 98
50 / 98
51 / 98
52 / 98
53 / 98
54 / 98
55 / 98
56 / 98
57 / 98
58 / 98
59 / 98
60 / 98
61 / 98
62 / 98
63 / 98
64 / 98
65 / 98
66 / 98
67 / 98
68 / 98
69 / 98
70 / 98
71 / 98
72 / 98
73 / 98
74 / 98
75 / 98
76 / 98
77 / 98
78 / 98
79 / 98
80 / 98
81 / 98
82 / 98
83 / 98
84 / 98
85 / 98
86 / 98
87 / 98
88 / 98
89 / 98
90 / 98
91 / 98
92 / 98
93 / 98
94 / 98
95 / 98
96 / 98
97 / 98


In [36]:
orders_df[orders_df['id']=='xnDoY1pLDDxYIwuODHUYWdBmsjEZY']

,id,created_at,customer_id,state,first_name,last_name,variation_name,item_name,note,total_money,customer_name
4,xnDoY1pLDDxYIwuODHUYWdBmsjEZY,2023-09-02T09:41:07.543Z,52AK01832BTM5GT4RVW8PRBYK0,OPEN,Jason,Heath,None,Match Subs - Half Game,None,5.0,Jason Heath


# Get Correct columns and filter out unwanted orders

In [37]:
orders_df['customer_name']= orders_df['first_name'] + ' ' + orders_df['last_name']
match_fees_export = orders_df[orders_df['state'].isin(['COMPLETED','OPEN'])][['id', 'created_at', 'variation_name','item_name', 'customer_name', 'total_money', 'note']]


match_fees_export = match_fees_export[(match_fees_export['item_name'].fillna('').str.lower().str.contains('sub'))|(match_fees_export['item_name'].fillna('').str.lower().str.contains('training'))]

In [38]:
match_fees_export.columns = exist.columns

In [39]:
match_fees_export

,ID,Date,Cat,Item,Customer,Price,Notes
0,V3AZRX79S94Y9HfeFsEjFzvIZ9YZY,2023-09-01T13:17:42.556Z,Regular,Training Sub,James Timmins,5.0,None
1,lvMZa4uaBX32eXnziIdUEcwBxBKZY,2023-09-01T16:50:25.231Z,Regular,Training Sub,NaN,5.0,None
4,xnDoY1pLDDxYIwuODHUYWdBmsjEZY,2023-09-02T09:41:07.543Z,None,Match Subs - Half Game,Jason Heath,5.0,None
7,Rb9AoGzQjZMxbnGZLFj8nQmYyZ8YY,2023-09-02T12:52:55.831Z,None,Match Subs,Masood Hamidi,10.0,None
14,r7Tn4WXctsjAqyYGo45Eoa6eV,2023-09-02T13:05:07Z,U23 Full Game,Match Sub,Ellis Forrester,5.0,None
18,PvGKJ2s6S4BWzEZ9lmD8TZjeV,2023-09-02T13:06:06Z,Half Game,Match Sub,Christian King,5.0,None
19,HL2yvFJZ5HOJIrCNOFRwpejeV,2023-09-02T13:06:41Z,Half Game,Match Sub,Niall Wootten,5.0,None
22,tmI1PGSHDRkmDCLW1xBJCHreV,2023-09-02T13:07:35Z,Full Game,Match Sub,Pearl Bakengela,10.0,None
23,lmom2FwJwurIIH0c1mGYjaxeV,2023-09-02T13:08:29Z,Half Game,Match Sub,Charlie Brooks,5.0,None
30,370DhMfPepGgTVRDDVbHf0peV,2023-09-02T13:11:48Z,U23 Full Game,Match Sub,Cameron Hildebrandt-Brooks,5.0,None


# Only upload new transactions

In [40]:
mg = pd.merge(left=match_fees_export, right=exist, how='left', on='ID', suffixes=['', '_old'])
not_in = mg[mg['Date_old'].isnull()][exist.columns]
len(not_in)

41

In [41]:
not_in.fillna('NA', inplace=True)

# Add to Google sheet

In [42]:
append_to_sheet('Square Subs', SAMPLE_SPREADSHEET_ID_input, not_in)